In [1]:
import pandas as pd
import numpy as np

In [2]:
#load data

df = pd.read_csv("consumer_complaints_response.csv")

C:\Users\my pc\AppData\Local\Temp\ipykernel_9048\3101122989.py:3: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("consumer_complaints_response.csv")


In [3]:
df.head()

,date_received,product,sub_product,issue,sub_issue,consumer_complaint_narrative,company_public_response,company,state,zipcode,tags,consumer_consent_provided,submitted_via,date_sent_to_company,company_response_to_consumer,timely_response,consumer_disputed?,complaint_id
0,8/30/2013,Mortgage,Other mortgage,"Loan modification,collection,foreclosure",NaN,NaN,NaN,U.S. Bancorp,CA,95993,NaN,NaN,Referral,9/3/2013,Closed with explanation,Yes,Yes,511074.0
1,8/30/2013,Mortgage,Other mortgage,"Loan servicing, payments, escrow account",NaN,NaN,NaN,Wells Fargo & Company,CA,91104,NaN,NaN,Referral,9/3/2013,Closed with explanation,Yes,Yes,511080.0
2,8/30/2013,Credit reporting,NaN,Incorrect information on credit report,Account status,NaN,NaN,Wells Fargo & Company,NY,11764,NaN,NaN,Postal mail,9/18/2013,Closed with explanation,Yes,No,510473.0
3,8/30/2013,Student loan,Non-federal student loan,Repaying your loan,Repaying your loan,NaN,NaN,"Navient Solutions, Inc.",MD,21402,NaN,NaN,Email,8/30/2013,Closed with explanation,Yes,Yes,510326.0
4,8/30/2013,Debt collection,Credit card,False statements or representation,Attempted to collect wrong amount,NaN,NaN,Resurgent Capital Services L.P.,GA,30106,NaN,NaN,Web,8/30/2013,Closed with explanation,Yes,Yes,511067.0


In [4]:
print(df.info())         

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 555957 entries, 0 to 555956
Data columns (total 18 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   date_received                 199999 non-null  object 
 1   product                       199999 non-null  object 
 2   sub_product                   144059 non-null  object 
 3   issue                         199999 non-null  object 
 4   sub_issue                     96010 non-null   object 
 5   consumer_complaint_narrative  550 non-null     object 
 6   company_public_response       1145 non-null    object 
 7   company                       199999 non-null  object 
 8   state                         198967 non-null  object 
 9   zipcode                       199342 non-null  object 
 10  tags                          29321 non-null   object 
 11  consumer_consent_provided     1367 non-null    object 
 12  submitted_via                 199999 non-nul

In [5]:
df.columns

Index(['date_received', 'product', 'sub_product', 'issue', 'sub_issue',
       'consumer_complaint_narrative', 'company_public_response', 'company',
       'state', 'zipcode', 'tags', 'consumer_consent_provided',
       'submitted_via', 'date_sent_to_company', 'company_response_to_consumer',
       'timely_response', 'consumer_disputed?', 'complaint_id'],
      dtype='object')

In [6]:
issues = df['consumer_complaint_narrative'].unique       #unique() gives us array of all rows
print(issues)

<bound method Series.unique of 0         NaN
1         NaN
2         NaN
3         NaN
4         NaN
         ... 
555952    NaN
555953    NaN
555954    NaN
555955    NaN
555956    NaN
Name: consumer_complaint_narrative, Length: 555957, dtype: object>


In [7]:
unique_issues_count = df['consumer_complaint_narrative'].nunique()
print(unique_issues_count)

546


In [8]:
# 546 kind of different issues are there faced by the users

In [9]:
category_wise_count = df['consumer_complaint_narrative'].value_counts()
print(category_wise_count)

consumer_complaint_narrative
This company continues to report on my credit report after I sent them a letter telling them that this account was not mine and I have no idea what it is or who it belongs to! \n\nI asked for proof of a signed contract, I asked for a license to collect in my state, I asked for copies of all information referenced for this debt and still to date, I have not received anything but harassment from this company! \n\nTHIS IS NOT MY DEBT! \n                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [10]:
print(df.isnull().count())

date_received                   555957
product                         555957
sub_product                     555957
issue                           555957
sub_issue                       555957
consumer_complaint_narrative    555957
company_public_response         555957
company                         555957
state                           555957
zipcode                         555957
tags                            555957
consumer_consent_provided       555957
submitted_via                   555957
date_sent_to_company            555957
company_response_to_consumer    555957
timely_response                 555957
consumer_disputed?              555957
complaint_id                    555957
dtype: int64


In [11]:
# Import libraries
import re
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score



In [12]:
# Download stopwords (run only once)
nltk.download('stopwords')
from nltk.corpus import stopwords



[nltk_data] Downloading package stopwords to C:\Users\my
[nltk_data]     pc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:

# Step 3: Select useful columns
df = df[["consumer_complaint_narrative", "timely_response"]].dropna()

# Step 4: Create synthetic resolution time (target variable)
# If company gave timely response → assume 24–72 hours
# If not timely → assume 72–144 hours
df["resolution_time"] = np.where(
    df["timely_response"] == "Yes",
    np.random.randint(24, 48, df.shape[0]),
    np.random.randint(72, 144, df.shape[0])
)

# Step 5: Clean text
stop_words = set(stopwords.words("english"))

def clean_text(text):
    text = text.lower()                              # lowercase
    text = re.sub(r'[^a-z\s]', '', text)             # remove special chars/numbers
    words = [w for w in text.split() if w not in stop_words]
    return " ".join(words)

df["clean_text"] = df["consumer_complaint_narrative"].apply(clean_text)

print("Cleaned text sample:")
print(df["clean_text"].head())

# Step 6: Convert text to numerical features (TF-IDF)
vectorizer = TfidfVectorizer(max_features=500)  # limit to top 500 words for speed
X = vectorizer.fit_transform(df["clean_text"])
y = df["resolution_time"]

# Step 7: Split into training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Step 8: Train regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Step 9: Make predictions
y_pred = model.predict(X_test)

# Step 10: Evaluate performance
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"\nModel Performance:")
print(f"Mean Absolute Error: {mae:.2f} hours")
print(f"R² Score: {r2:.2f}")






Cleaned text sample:
190126    xxxx claimed owe xxxx years despite proof paym...
190135    due inconsistencies amount owed told bank amou...
190155    xxxxxxxx wages earned job decreased almost hal...
190207    open current mortgage chase bank xxxx chase re...
190208    xxxx submitted xxxxxxxx time submitted complai...
Name: clean_text, dtype: object

Model Performance:
Mean Absolute Error: 95.85 hours
R² Score: -85.48


In [14]:
comparison = pd.DataFrame({
    "Complaint Text": df.loc[y_test.index, "clean_text"].values[:10],
    "Actual (hrs)": y_test.values[:10],
    "Predicted (hrs)": np.round(y_pred[:10], 2)
})

print("\nSample Predictions with Complaint Text:\n")
print(comparison)


Sample Predictions with Complaint Text:

                                      Complaint Text  Actual (hrs)  \
0  contacted select portfolio mortgage servicing ...            36   
1  started dealing citi mortgage back xxxxxxxx mo...            47   
2  debt collector portfolio recovery phone number...            29   
3  nationstar denied loan modification property p...            41   
4  originally owed xxxx xxxx credit card point re...            43   
5  received letter xxxx debit card representing r...            29   
6  mortgaged home xxxx xxxxxxxxxxxx xxxx assigned...            41   
7  attended xxxx xxxx xxxx xxxx took private loan...            36   
8  wife laid xxxx xxxx xxxx modification request ...            42   
9  current mortgage chase home finance excellent ...            40   

   Predicted (hrs)  
0            28.45  
1           -81.15  
2            57.71  
3            36.28  
4           275.27  
5            -8.35  
6            21.17  
7           153.54 

In [17]:
#User Input Prediction
print("\n--- Write your complaint ---")
user_input = input("Enter your complaint description: ")

# Clean the input same way as training data
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    words = [w for w in text.split() if w not in stop_words]
    return " ".join(words)

cleaned_input = clean_text(user_input)

# Convert to TF-IDF vector using the same vectorizer
input_vector = vectorizer.transform([cleaned_input])

# Predict resolution time
predicted_time = model.predict(input_vector)[0]

print(f"\nEstimated Resolution Time: {predicted_time:.2f} hours")



--- Write your complaint ---


Enter your complaint description:  This company continues to report on my credit report after I sent them a letter telling them that this account was not mine and I have no idea what it is or who it belongs to! \n\nI asked for proof of a signed contract, I asked for a license to collect in my state, I asked for copies of all information referenced for this debt and still to date, I have not received anything but harassment from this company! \n\nTHIS IS NOT MY DEBT!



Estimated Resolution Time: 33.37 hours
